# Preparación Pre DB

Vamos a preparar los DataFrame para insertarlos en la Base de datos

In [1]:
# Importamos las librerías que necesitamos

# Librerías para automatización de navegadores (Selenium)
# -----------------------------------------------------------------------
from selenium import webdriver                   # Control automático del navegador para realizar scraping web

from selenium.webdriver.chrome.service import Service  # Maneja el servicio de ChromeDriver
from selenium.webdriver.common.by import By      # Para seleccionar elementos en la página web (DOM)
from selenium.webdriver.chrome.options import Options  # Configuración de opciones del navegador (headless mode, etc.)
import time                                      # Para gestionar pausas y temporización

# Librerías para extracción de datos y scraping
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup  # Para analizar y extraer datos de HTML y XML (scraping web)

# Librerías para tratamiento y manipulación de datos
# -----------------------------------------------------------------------
import pandas as pd             # Para manipulación de estructuras de datos como DataFrames
import numpy as np              # Para cálculos numéricos y manejo de arrays
import datetime                 # Para manipulación de fechas y tiempos
# -----------------------------------------------------------------------
# Librerías para visualización de progreso
# -----------------------------------------------------------------------
from tqdm import tqdm  # Para crear barras de progreso durante los bucles

# Manejo de tiempo y solicitudes con espera
# -----------------------------------------------------------------------
from time import sleep          # Para pausar la ejecución del código por un tiempo determinado
import random                   # Para generar valores aleatorios, útil para espaciar solicitudes y evitar bloqueos
# Importación de módulos desde rutas recursivas
# -----------------------------------------------------------------------
import sys
sys.path.append("../")  # Añadir rutas externas para poder importar módulos desde carpetas superiores
from src.SupportPreDB import crear_df_unicos, crear_df_historico

# Abrir tabla
Para continuar donde lo dejamos en la Fase 1

In [2]:
df_historicos =  pd.read_csv("../datos/01_tabla_historicos_preDB.csv",index_col="Unnamed: 0")

In [3]:
df_historicos.sample()

,supermercado,categoria,producto,fecha,precio (€),variacion (€),variacion (%)
46210,dia,aceite de oliva virgen extra,Aceite De Oliva Virgen Extra Ecológico Oro De ...,2024-10-05,7.76,0.0,0.0


Ver si merece la pena tener una tabla producto

Con 1523 productos, nos merece la pena para no repetirlos en los 128796

In [4]:
len(df_historicos["producto"].unique())

1523

# Renombrar una categoría

Por claridad renombro la categoría "leche entera semi desnatada" por "leche entera, semi o desnatada"

In [5]:
df_historicos["categoria"] =df_historicos["categoria"].str.replace("leche entera semi desnatada",'leche entera, semi o desnatada')


### Separar las tablas para realizar la base de Datos
Tendremos 4 tablas:
- Supermercado
- Categoría
- Producto
- Histórico

En la fase 3 detallaremos sus columnas y demás, por ahora solo queremos preparar los DataFrames

Vamos a realizar lo siguiente:
- Separar Supermercado, Categoría y Producto, cada uno en un DataFrame individual, con dos columnas, un index que empieza en uno, que compatibiliza con el funcionamiento de los índices en SQL y otra con los valores únicos de cada una de las columnas.

- DF histórico con índices de Supermercado, categoría y Producto: Generamos un diccionario para cada una de las tablas anteriores, con ello lo utilizaremos para reemplazar los valores de esas columnas por el índice que tendran en la Base de Datos, lo que nos ayudará a vincular las Foreign Keys

### Separar Supermercados

In [6]:
df_supermercados = crear_df_unicos(df_historicos,"supermercado")

In [7]:
df_supermercados.sample()

,index,supermercado
2,3,eroski


### Guardamos la tabla que ya está realizada

In [8]:
df_supermercados.to_csv("../datos/02_tabla_supermercados.csv")

### Separar Categoría

In [9]:
df_categorias = crear_df_unicos(df_historicos,"categoria")

In [10]:
df_categorias.sample()

,index,categoria
1,2,aceite de oliva suave e intenso


### Guardamos la tabla que ya está realizada

In [11]:
df_categorias.to_csv("../datos/03_tabla_categorias.csv")

### Separar Producto

In [12]:
df_productos = crear_df_unicos(df_historicos,"producto")

In [13]:
df_productos.sample()

,index,producto
1403,1404,"Laban Leche Fresca Parcialmente Desnatada, Pas..."


### Guardamos la tabla que ya está realizada

In [14]:
df_productos.to_csv("../datos/04_tabla_productos.csv")

# Preparar DF Histórico

Con las tablas que tenemos ahora, realizaremos los diccionarios y reemplazaremos en nuestra tabla original, los valores de los mismos por sus respectivos índices


In [15]:
df_historico = crear_df_historico(df_historicos,df_supermercados,df_categorias,df_productos)

In [18]:
df_historico.sample(5)

,supermercado,categoria,producto,fecha,precio (€),variacion (€),variacion (%)
14076,2,5,151,2024-08-03,0.93,0.0,0.0
24226,3,3,323,2024-07-18,41.70,0.0,0.0
120688,6,6,1425,2024-08-24,1.13,0.0,0.0
81347,5,6,969,2024-08-27,2.37,0.0,0.0
16742,2,6,206,2024-10-06,1.16,0.0,0.0


### Guardamos el resultado y pasamos a Fase 3

In [19]:
df_historico.to_csv("../datos/05_tabla_historico.csv")